# Timeseries Cost
*TS utilities in Qiskit*

By: Jacob Cybulski<br>
Date: August 2023 - April 2024<br>
Aims: The goal of this notebook is to test some of the QuTSAE cost related calculations.

## Initial settings

In [1]:
import sys
sys.path.append('.')
sys.path.append('..')
sys.path

['/home/jacob/Dropbox/Dev/Python/quantum_projects/ts/ts_denoise/utils',
 '/home/jacob/miniconda3/envs/qiskit-gpu/lib/python310.zip',
 '/home/jacob/miniconda3/envs/qiskit-gpu/lib/python3.10',
 '/home/jacob/miniconda3/envs/qiskit-gpu/lib/python3.10/lib-dynload',
 '',
 '/home/jacob/miniconda3/envs/qiskit-gpu/lib/python3.10/site-packages',
 '.',
 '..']

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pylab
import math
from IPython.display import clear_output

# from utils.Target import *
# from utils.Window import *
# from utils.Callback import *
# from utils.TS import *
# from utils.Charts import *
# from utils.Integrate import *
# from utils.Circuits import *
# from utils.Angles import *
# from utils.Files import *
from utils.Cost import *
# from utils.Metrics import *

from matplotlib import set_loglevel
set_loglevel("error")

%matplotlib inline

## Measuring the states of single qubits in a circuit (in terms of its y angle)
Functions "single_qubit_angle_meas" and "multi_qubit_angle_meas" are defined in utils.Cost.<br/>
Recall that the value returned is in relation to the H state, which is zero,<br/>
with rotations to the left interpretaed as negative values, and rotatations to the right as positive values.

In [5]:
from qiskit_aer.backends import AerSimulator
from qiskit import Aer, execute
# noise_model = NoiseModel.from_backend(backend)
seed = 2024
backend = AerSimulator(method='statevector')
backend.set_option('device', 'CPU')
backend.set_option('seed_simulator', seed)
backend.set_option('cuStateVec_enable', True)

In [6]:
### Testing cprobs_to_qprobs
probs1 = [0.1, 0.05, 0.05, 0.3, 0.2, 0.1, 0.2, 0.0]
probs2 = [0.01, 0.02, 0.3, 0, 0.03, 0, 0.1, 0, 0.04, 0, 0.05, 0.05, 0.1, 0.2, 0.02, 0.08]

res = cprobs_to_qprobs(probs2)
print('Result of mapping a circuit state to\nindividual qubit states:\n\n\t', res, '\n')

report_c2p(probs2)

Result of mapping a circuit state to
individual qubit states:

	 [0.54 0.53 0.6  0.35] 



array([[0.  , 0.  , 0.  , 0.  , 0.01],
       [0.  , 0.  , 0.  , 1.  , 0.02],
       [0.  , 0.  , 1.  , 0.  , 0.3 ],
       [0.  , 0.  , 1.  , 1.  , 0.  ],
       [0.  , 1.  , 0.  , 0.  , 0.03],
       [0.  , 1.  , 0.  , 1.  , 0.  ],
       [0.  , 1.  , 1.  , 0.  , 0.1 ],
       [0.  , 1.  , 1.  , 1.  , 0.  ],
       [1.  , 0.  , 0.  , 0.  , 0.04],
       [1.  , 0.  , 0.  , 1.  , 0.  ],
       [1.  , 0.  , 1.  , 0.  , 0.05],
       [1.  , 0.  , 1.  , 1.  , 0.05],
       [1.  , 1.  , 0.  , 0.  , 0.1 ],
       [1.  , 1.  , 0.  , 1.  , 0.2 ],
       [1.  , 1.  , 1.  , 0.  , 0.02],
       [1.  , 1.  , 1.  , 1.  , 0.08]])

In [7]:
### Testing single_qubit_angle_meas

from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram

start = time.time()
q_meas =[]
q_ps = []
for q in range(3):
    qr = QuantumRegister(3, 'q')
    anc = QuantumRegister(1, 'ancilla')
    cr = ClassicalRegister(1, 'c')
    qc = QuantumCircuit(qr, cr)

    qc.ry(np.pi/6, 0)
    qc.ry(-np.pi/4, 1)
    qc.ry(np.pi/3, 2)
    qc.save_statevector()
    qc.measure(q, cr)

    display(qc.draw())

    ma, p0 = single_qubit_angle_meas(qc, backend, shots=1000)
    q_meas.append(ma) 
    q_ps.append(p0)

elapsed = time.time() - start

print('Q meas:', q_meas)
print('Probs:', q_ps)
print('Elapsed time:', elapsed)

┌─────────┐  statevector ┌─┐
q_0: ┤ Ry(π/6) ├───────░──────┤M├
     ├─────────┴┐      ░      └╥┘
q_1: ┤ Ry(-π/4) ├──────░───────╫─
     ├─────────┬┘      ░       ║ 
q_2: ┤ Ry(π/3) ├───────░───────╫─
     └─────────┘       ░       ║ 
c: 1/══════════════════════════╩═
                               0

┌─────────┐  statevector    
q_0: ┤ Ry(π/6) ├───────░─────────
     ├─────────┴┐      ░      ┌─┐
q_1: ┤ Ry(-π/4) ├──────░──────┤M├
     ├─────────┬┘      ░      └╥┘
q_2: ┤ Ry(π/3) ├───────░───────╫─
     └─────────┘       ░       ║ 
c: 1/══════════════════════════╩═
                               0

┌─────────┐  statevector    
q_0: ┤ Ry(π/6) ├───────░─────────
     ├─────────┴┐      ░         
q_1: ┤ Ry(-π/4) ├──────░─────────
     ├─────────┬┘      ░      ┌─┐
q_2: ┤ Ry(π/3) ├───────░──────┤M├
     └─────────┘       ░      └╥┘
c: 1/══════════════════════════╩═
                               0

Q meas: [-1.0046979448455693, -0.783834160846974, -0.5351847902755997]
Probs: [0.922, 0.853, 0.755]
Elapsed time: 0.006272077560424805


In [8]:
### Testing multi_qubit_angle_meas

from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram

start = time.time()

qr = QuantumRegister(3, 'q')
cr = ClassicalRegister(3, 'c')
qc = QuantumCircuit(qr, cr)
 
qc.ry(np.pi/6, 0)
qc.ry(-np.pi/4, 1)
qc.ry(np.pi/3, 2)
# qc.cx(0, 1)
# qc.cx(1, 2)
qc.save_statevector()
qc.measure(qr, cr)
display(qc.draw())

q_meas, probs = multi_qubit_angle_meas(qc, backend=backend, shots=1)

elapsed = time.time() - start

print('Q meas:', q_meas)
print('Probs:', probs)
print('Elapsed time:', elapsed)

┌─────────┐  statevector ┌─┐      
q_0: ┤ Ry(π/6) ├───────░──────┤M├──────
     ├─────────┴┐      ░      └╥┘┌─┐   
q_1: ┤ Ry(-π/4) ├──────░───────╫─┤M├───
     ├─────────┬┘      ░       ║ └╥┘┌─┐
q_2: ┤ Ry(π/3) ├───────░───────╫──╫─┤M├
     └─────────┘       ░       ║  ║ └╥┘
c: 3/══════════════════════════╩══╩══╩═
                               0  1  2

Q meas: [-1.0471975511965979, -0.7853981633974482, -0.5235987755982987]
Probs: [0.9330127018922193, 0.8535533905932737, 0.75]
Elapsed time: 0.006639242172241211


## System in use

In [9]:
!pip list | grep -e torch -e qiskit

qiskit                        0.45.2
qiskit-aer-gpu                0.13.2
qiskit-algorithms             0.2.2
qiskit-dynamics               0.4.3
qiskit-finance                0.4.0
qiskit-ibm-provider           0.6.3
qiskit-ibm-runtime            0.11.3
qiskit-ibmq-provider          0.20.2
qiskit-machine-learning       0.7.1
qiskit-nature                 0.7.1
qiskit-optimization           0.6.0
qiskit-qasm3-import           0.4.1
qiskit-sphinx-theme           1.12.1
qiskit-terra                  0.45.2
torch                         2.1.0
torchaudio                    2.1.0
torchsummary                  1.5.1
torchvision                   0.16.0
torchviz                      0.0.2
